# GLOBAL UNDERNUTRITION STUDY - EXPLORATION  

*BY FURAWA*  

**Table of Contents**  

1. [Data collection](#data_collection)  
2. [Data discovery](#data_discovery)  
3. [Data cleaning](#data_cleaning)  
4. [Computing new variables to lead the analysis](#new_variables)  
5. [Identify major trends](#major_trends)  

In [1]:
# Import all the needed libraries for the project
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
import re
%matplotlib inline
pd.set_option('max_rows', 20)

<a id='data_collection'></a>
## 1. Data collection  
All the data has been downloaded from the [FAO](http://www.fao.org/faostat/en/#data) website.  
Let us check the files.

In [2]:
file_names = glob('files/*.csv')
file_names

['files/food_security_indicators.csv',
 'files/commodity_balance_crops.csv',
 'files/food_supply_crops.csv',
 'files/commodity_balance_livestock.csv',
 'files/food_balance_sheet.csv',
 'files/food_supply_livestock.csv']

In [8]:
for file in file_names:
    exec(re.split('\. |\W', file)[1] + "=  pd.read_csv(file)")

In [18]:
food_security_indicators.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3173 entries, 0 to 3172
Data columns (total 15 columns):
Domain Code         3173 non-null object
Domain              3173 non-null object
Area Code           3173 non-null int64
Area                3173 non-null object
Element Code        3173 non-null int64
Element             3173 non-null object
Item Code           3173 non-null int64
Item                3173 non-null object
Year Code           3173 non-null int64
Year                3173 non-null object
Unit                3173 non-null object
Value               3052 non-null object
Flag                3173 non-null object
Flag Description    3173 non-null object
Note                0 non-null float64
dtypes: float64(1), int64(4), object(10)
memory usage: 372.0+ KB


In [19]:
food_supply_crops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60628 entries, 0 to 60627
Data columns (total 14 columns):
Domain Code         60628 non-null object
Domain              60628 non-null object
Country Code        60628 non-null int64
Country             60628 non-null object
Element Code        60628 non-null int64
Element             60628 non-null object
Item Code           60628 non-null int64
Item                60628 non-null object
Year Code           60628 non-null int64
Year                60628 non-null int64
Unit                60628 non-null object
Value               60628 non-null float64
Flag                60628 non-null object
Flag Description    60628 non-null object
dtypes: float64(1), int64(5), object(8)
memory usage: 6.5+ MB


In [20]:
food_supply_livestock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25569 entries, 0 to 25568
Data columns (total 14 columns):
Domain Code         25569 non-null object
Domain              25569 non-null object
Country Code        25569 non-null int64
Country             25569 non-null object
Element Code        25569 non-null int64
Element             25569 non-null object
Item Code           25569 non-null int64
Item                25569 non-null object
Year Code           25569 non-null int64
Year                25569 non-null int64
Unit                25569 non-null object
Value               25569 non-null float64
Flag                25569 non-null object
Flag Description    25569 non-null object
dtypes: float64(1), int64(5), object(8)
memory usage: 2.7+ MB


In [22]:
food_balance_sheet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142212 entries, 0 to 142211
Data columns (total 14 columns):
Domain Code         142212 non-null object
Domain              142212 non-null object
Country Code        142212 non-null int64
Country             142212 non-null object
Element Code        142212 non-null int64
Element             142212 non-null object
Item Code           142212 non-null int64
Item                142212 non-null object
Year Code           142212 non-null int64
Year                142212 non-null int64
Unit                142212 non-null object
Value               142212 non-null float64
Flag                142038 non-null object
Flag Description    142212 non-null object
dtypes: float64(1), int64(5), object(8)
memory usage: 15.2+ MB


In [24]:
commodity_balance_crops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85123 entries, 0 to 85122
Data columns (total 14 columns):
Domain Code         85123 non-null object
Domain              85123 non-null object
Country Code        85123 non-null int64
Country             85123 non-null object
Element Code        85123 non-null int64
Element             85123 non-null object
Item Code           85123 non-null int64
Item                85123 non-null object
Year Code           85123 non-null int64
Year                85123 non-null int64
Unit                85123 non-null object
Value               85123 non-null int64
Flag                85123 non-null object
Flag Description    85123 non-null object
dtypes: int64(6), object(8)
memory usage: 9.1+ MB


In [25]:
commodity_balance_livestock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30814 entries, 0 to 30813
Data columns (total 14 columns):
Domain Code         30814 non-null object
Domain              30814 non-null object
Country Code        30814 non-null int64
Country             30814 non-null object
Element Code        30814 non-null int64
Element             30814 non-null object
Item Code           30814 non-null int64
Item                30814 non-null object
Year Code           30814 non-null int64
Year                30814 non-null int64
Unit                30814 non-null object
Value               30814 non-null int64
Flag                30814 non-null object
Flag Description    30814 non-null object
dtypes: int64(6), object(8)
memory usage: 3.3+ MB
